In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.005)
    q3 = df_in[col_name].quantile(0.995)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out


In [ ]:
df = pd.read_csv('/content/Salaries.csv')
df = remove_outlier(df, 'yearsExperience')
df = remove_outlier(df, 'milesFromMetropolis')
df = remove_outlier(df, 'salary')

In [ ]:
x_columns = df.columns
# x_columns

In [ ]:
x_columns = x_columns.drop('salary')

In [ ]:
X = df[x_columns]
Y = df['salary']

In [ ]:
X

,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,CFO,MASTERS,MATH,HEALTH,10,83
1,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16
...,...,...,...,...,...,...
999995,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,19,94
999996,CTO,HIGH_SCHOOL,NONE,FINANCE,12,35
999997,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,16,81
999998,CFO,MASTERS,NONE,HEALTH,6,5


In [ ]:
X.columns

Index(['jobType', 'degree', 'major', 'industry', 'yearsExperience',
       'milesFromMetropolis'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Label Encoding Data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['jobType']=le.fit_transform(X['jobType'])
X['degree']=le.fit_transform(X['degree'])
X['major']=le.fit_transform(X['major'])
X['industry']=le.fit_transform(X['industry'])
X.head()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)
X_train_all = scaling.transform(X)

In [ ]:
# create an ann
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
# import early stopping
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=5)

regressor = Sequential()
regressor.add(Dense(units = 256, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))
regressor.add(Dropout(0.5))
regressor.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(0.5))
regressor.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(0.5))
regressor.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(0.5))
regressor.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [ ]:
# create a regressor object

# train the model
regressor.fit(X_train, y_train, batch_size = 32, epochs = 15, callbacks = [es])

# predict the result
y_pred = regressor.predict(X_test)

# evaluate the model
from sklearn import metrics
# rmse

Epoch 1/15
24992/25000 [============================>.] - ETA: 0s - loss: 1496.6235 - accuracy: 0.0000e+00

25000/25000 [==============================] - 85s 3ms/step - loss: 1496.4867 - accuracy: 0.0000e+00
Epoch 2/15
24989/25000 [============================>.] - ETA: 0s - loss: 942.2173 - accuracy: 0.0000e+00

25000/25000 [==============================] - 82s 3ms/step - loss: 942.1985 - accuracy: 0.0000e+00
Epoch 3/15
24991/25000 [============================>.] - ETA: 0s - loss: 831.3221 - accuracy: 0.0000e+00

25000/25000 [==============================] - 83s 3ms/step - loss: 831.2891 - accuracy: 0.0000e+00
Epoch 4/15
24995/25000 [============================>.] - ETA: 0s - loss: 746.5990 - accuracy: 0.0000e+00

25000/25000 [==============================] - 79s 3ms/step - loss: 746.5757 - accuracy: 0.0000e+00
Epoch 5/15
24987/25000 [============================>.] - ETA: 0s - loss: 690.9114 - accuracy: 0.0000e+00

...

25000/25000 [==============================] - 74s 3ms/step - loss: 631.0329 - accuracy: 0.0000e+00
Epoch 12/15
24996/25000 [============================>.] - ETA: 0s - loss: 628.4572 - accuracy: 0.0000e+00

25000/25000 [==============================] - 76s 3ms/step - loss: 628.4480 - accuracy: 0.0000e+00
Epoch 13/15
24991/25000 [============================>.] - ETA: 0s - loss: 624.6660 - accuracy: 0.0000e+00

25000/25000 [==============================] - 74s 3ms/step - loss: 624.7162 - accuracy: 0.0000e+00
Epoch 14/15
24983/25000 [============================>.] - ETA: 0s - loss: 616.2395 - accuracy: 0.0000e+00

25000/25000 [==============================] - 73s 3ms/step - loss: 616.2586 - accuracy: 0.0000e+00
Epoch 15/15
24997/25000 [============================>.] - ETA: 0s - loss: 603.2796 - accuracy: 0.0000e+00

6250/6250 [==============================] - 9s 1ms/step


In [ ]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 20.638452561282193


In [ ]:
print(regressor.layers[4].weights[0])

<tf.Variable 'dense_2/kernel:0' shape=(128, 64) dtype=float32, numpy=
array([[-5.5260806e+00, -5.7013124e-01, -6.1893196e+00, ...,
        -6.4014358e-04, -7.5050111e+00, -5.9181457e+00],
       [ 2.4811324e-01,  1.3421947e-01,  1.3663459e-01, ...,
         3.0385661e-01,  1.9642732e-01,  1.6465688e-01],
       [-1.6788903e-01,  1.1362920e-01, -9.6190929e-02, ...,
         9.8242126e-02, -1.9561081e-01,  6.7202352e-02],
       ...,
       [ 5.5115563e-03,  3.1833807e-01, -2.0622951e-01, ...,
        -2.5032062e-02,  7.3950350e-02,  6.5318331e-02],
       [-1.2971170e+01, -5.1890700e-03, -3.8364110e+00, ...,
         9.6062161e-03, -1.1218014e+01, -5.5086589e+00],
       [-9.4006695e-02,  1.9575921e-01,  6.1961770e-02, ...,
         1.3740568e-01,  3.1709874e-01,  6.5459572e-02]], dtype=float32)>
